In [261]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as wg
from scipy.signal import find_peaks
import sys
sys.path.append("../src")
import importlib
import support
import visualizations
import calibrations
import kinematics
from scipy import signal
importlib.reload(support)
from support import readFileDialog, parse_data_file, getFrame, writeToDATA, saveFileDialog
importlib.reload(kinematics)
from kinematics import getKeypointsCoord, getKeypointsPixels, inverseKinematicsRowing, getRowingAngles
importlib.reload(visualizations)
from visualizations import showFrame, keypointsDATAtoFrame, keypointsToVideoCrop
importlib.reload(calibrations)
from calibrations import getMmppInterface, defineReference
%matplotlib widget

In [262]:
file_path_op = readFileDialog("Open Data file (OP)", "data")

In [263]:
file_path_op

''

In [264]:
data, var_names = parse_data_file(file_path_op)
kp_openpose = data["keypoints"]
md_openpose = data["metadata"]
ang_openpose = data["angles"]
fps_openpose = md_openpose["fps"]
angles_names = md_openpose['angles_names']
t_openpose = np.linspace(0, len(ang_openpose)*(1/fps_openpose), len(ang_openpose))

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
ang_openpose_corr = getRowingAngles(kp_openpose)

In [ ]:
ang_openpose_corr[:, 3] = -ang_openpose_corr[:, 3]

In [ ]:
joint_angle = wg.Dropdown(options=angles_names,
                        description='Angle:',
                        value='Shoulder <- Hip -> Knee',
                        disabled=False)

def plot_2d_comparison(joint_angle):
    angle_idx = angles_names.index(joint_angle)
    plt.close('all')
    plt.figure()
    plt.plot(t_openpose, ang_openpose_corr[:, angle_idx], label='Corrected')
    plt.plot(t_openpose, ang_openpose[:, angle_idx],  label='Opened')
    angle_name = joint_angle.split('->')[0].split('<-')[-1].replace(" ", "")
    plt.title("2D Comparation for the {}".format(angle_name))
    plt.xlabel("Time (s)")
    plt.ylabel("Angle (Degrees)")
    plt.grid(True)
    plt.legend()
    plt.show()

out = wg.interactive_output(plot_2d_comparison, {'joint_angle': joint_angle})

vbox = wg.VBox([joint_angle, out])
display(vbox)

In [ ]:
output_path = saveFileDialog()
output_path

In [ ]:
writeToDATA(output_path, md_openpose, write_mode='w')
for i in range(md_openpose["n_frames"]):
    file_data = {'keypoints': kp_openpose[i].tolist(),
                 'angles': ang_openpose_corr[i].tolist()}
    writeToDATA(output_path, file_data, write_mode='a')

In [ ]:
md_openpose_crop = md_openpose.copy()

In [ ]:
t_sec = 5
md_openpose_crop["n_frames"] = md_openpose["fps"]*t_sec

In [ ]:
starting_t_sec = 150
starting_frame = np.argmin(np.abs(t_openpose - 150))

In [ ]:
kp_openpose_crop = kp_openpose[starting_frame:starting_frame+md_openpose_crop["n_frames"]]
ang_openpose_crop = ang_openpose_corr[starting_frame:starting_frame+md_openpose_crop["n_frames"]]

In [ ]:
output_path_5_sec = output_path.split(".")[0] + f"_{t_sec}seg." + output_path.split(".")[-1]
output_path_5_sec

In [ ]:
plt.figure()
plt.plot(ang_openpose_crop[:, 2])
plt.show()

In [ ]:
plt.figure()
plt.plot(ang_openpose_crop[:, 2])
plt.show()

In [ ]:
writeToDATA(output_path_5_sec, md_openpose_crop, write_mode='w')
for i in range(md_openpose_crop["n_frames"]):
    file_data = {'keypoints': kp_openpose_crop[i].tolist(),
                 'angles': ang_openpose_crop[i].tolist()}
    writeToDATA(output_path_5_sec, file_data, write_mode='a')

In [ ]:
video_path = file_path_op.split(".")[0] + ".mp4"
video_path

In [ ]:
video_out_path = output_path_5_sec.split(".")[0] + ".mp4"
video_out_path

In [ ]:
keypointsToVideoCrop(video_path, md_openpose_crop, kp_openpose_crop, video_out_path, starting_frame=int(md_openpose["n_frames"]/2))